In [1]:
from finviz.screener import Screener
import os
import datetime
import pandas as pd
import time
import yfinance as yf
from datetime import datetime
from datetime import timedelta

In [2]:
def fetch_stocks():
    filters = ['ind_exchangetradedfund','sh_avgvol_o2000']
    # filters = ['sh_avgvol_o2000', 'ind_stocksonly', 'cap_largeover', 'sh_price_o10']
    
    stock_list = Screener(filters=filters, table='Performance', order='price')
    
    return stock_list

In [3]:
def cal_VRRS(ticker, ref, rolling_length):
    ticker_sma = ticker.rolling(rolling_length).mean()
    ticker_change = ticker - ticker_sma
    
    ref_sma = ref.rolling(rolling_length).mean()
    ref_change = ref - ref_sma
    
    return (ticker_change/ticker_sma - ref_change/ref_sma) * 100

In [4]:
def value_to_float(x):
    if type(x) == float or type(x) == int:
        return x
    if 'K' in x:
        if len(x) > 1:
            return float(x.replace('K', '')) * 1000
        return 1000.0
    if 'M' in x:
        if len(x) > 1:
            return float(x.replace('M', '')) * 1000000
        return 1000000.0
    if 'B' in x:
        if len(x) > 1:
            return float(x.replace('B', '')) * 1000000000
        return 1000000000.0
    if 'T' in x:
        if len(x) > 1:
            return float(x.replace('T', '')) * 1000000000000
        return 1000000000000000.0

    return 0.0

In [5]:
ticker_dict = {}

In [6]:
stock_list = fetch_stocks()

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:04<00:00,  2.60it/s]


In [8]:
start_date = "2017-01-01"
# end_date = "2022-05-28"

end_date = datetime.today().date()

spy = yf.Ticker("SPY")
spy = spy.history(start=start_date, end=end_date, interval='5d')['Close'].dropna()

for stock in stock_list:
    
    ticker = stock['Ticker']
    
    avg_volume = value_to_float(stock['Avg Volume'])
    
    if avg_volume > 5000000:

        stock = yf.Ticker(ticker)    

        stock_close = stock.history(start=start_date, end=end_date, interval='5d')['Close'].dropna()
        stock_now = stock.history(start=start_date, end=end_date, interval='5d').dropna()

        # Calculate the VRRS indicator:
        rolling_length = 21 

        vrrs = cal_VRRS(stock_close, spy, rolling_length).iloc[-1]
        week_ago_VRRS = cal_VRRS(stock_close.iloc[:-1], spy.iloc[:-1], rolling_length).iloc[-1]

        difference = float(vrrs) - float(week_ago_VRRS)

        if abs(difference) > 4:
            print(ticker, ' || VRRS: ', round(vrrs, 2), '|| Week_Ago_VRRS: ', round(week_ago_VRRS, 2), ' || ', round(difference, 2))

LABU  || VRRS:  -11.09 || Week_Ago_VRRS:  -34.75  ||  23.65
UVXY  || VRRS:  -9.58 || Week_Ago_VRRS:  0.34  ||  -9.92
PSQ  || VRRS:  12.75 || Week_Ago_VRRS:  20.02  ||  -7.28
SH  || VRRS:  12.58 || Week_Ago_VRRS:  17.65  ||  -5.07
PDBC  || VRRS:  5.31 || Week_Ago_VRRS:  14.23  ||  -8.92
SLV  || VRRS:  -9.54 || Week_Ago_VRRS:  -1.61  ||  -7.93
SPXU  || VRRS:  18.96 || Week_Ago_VRRS:  29.82  ||  -10.86
UNG  || VRRS:  3.21 || Week_Ago_VRRS:  13.35  ||  -10.14
QID  || VRRS:  15.59 || Week_Ago_VRRS:  28.0  ||  -12.42
SPXS  || VRRS:  18.83 || Week_Ago_VRRS:  29.78  ||  -10.95
GDX  || VRRS:  -12.77 || Week_Ago_VRRS:  -6.78  ||  -5.98
GDXJ  || VRRS:  -17.75 || Week_Ago_VRRS:  -11.3  ||  -6.46
KWEB  || VRRS:  12.45 || Week_Ago_VRRS:  17.06  ||  -4.62
IAU  || VRRS:  -0.18 || Week_Ago_VRRS:  6.09  ||  -6.27
SDOW  || VRRS:  16.78 || Week_Ago_VRRS:  23.65  ||  -6.87
TNA  || VRRS:  -21.05 || Week_Ago_VRRS:  -25.66  ||  4.62
TZA  || VRRS:  17.47 || Week_Ago_VRRS:  31.03  ||  -13.56
XME  || VRRS:  -11.